# If files ae present start from here

In [1]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)

train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)


In [2]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((83000, 502), (10987, 502), (10988, 502))

In [3]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10987, 1])


In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import numpy as np

class CNN(nn.Module):
    def __init__(self,vocab_size,embed_size,filter_size,kernel_size,dropout,seq_len):
        super(CNN, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size,embed_size)
        self.conv1 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[0],embed_size])
        self.conv2 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[1],embed_size])
        self.conv3 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[2],embed_size])
        self.mp1 = torch.nn.MaxPool1d(seq_len+1-kernel_size[0])
        self.mp2 = torch.nn.MaxPool1d(seq_len+1-kernel_size[1])
        self.mp3 = torch.nn.MaxPool1d(seq_len+1-kernel_size[2])
        self.dense = torch.nn.Linear(filter_size*3,1)
        self.dropout = torch.nn.Dropout(dropout)
    
    def forward(self, x, target):
        embed_input = self.embedding(x)
        embed_input.unsqueeze_(1)
        x1 = torch.tanh(self.dropout(self.conv1(embed_input))).squeeze(3)
        x2 = torch.tanh(self.dropout(self.conv2(embed_input))).squeeze(3)
        x3 = torch.tanh(self.dropout(self.conv3(embed_input))).squeeze(3)
        f1 = self.mp1(x1).squeeze(2)
        f2 = self.mp2(x2).squeeze(2)
        f3 = self.mp3(x3).squeeze(2)
        hidden = torch.cat([f1,f2,f3],dim=1)
        logits = self.dense(hidden)
        prediction = torch.sigmoid(logits)
        target = target.view([-1,1])
        correct_pred = torch.eq(torch.round(prediction).type(target.type()),target)
        accuracy = torch.sum(correct_pred)
        return prediction, accuracy

In [5]:
# write to file.
filename = "CNN_amazon.csv"
def write_to_csv(epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 10
batch_size = 50
learning_rate = 0.001
embed_size = 300
seq_len = 500
dropout = 0.5
filter_size = 100
vocab_size = 10000
embed_dims = 32
kernel_size = [3,4,5]
model = CNN(vocab_size,embed_dims,filter_size,kernel_size,dropout,seq_len).to(device)

In [9]:
import time
import numpy as np

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
counter = 0
CNN_acc = []
CNN_valacc = []
train_loss_epoch = []
train_acc_epoch = []
val_loss_epoch = []
val_acc_epoch = []
time_epoch = []
model.train()
for e in range(epochs):
    start_time = time.time()
    train_loss = []
    train_acc = []
    for inputs, labels in train_dl:
        inputs, labels = inputs.cuda(), labels.cuda()
        model.zero_grad()
        logits, accuracy = model(inputs,labels)
        loss = criterion(logits,labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        train_loss.append(loss.item())
        train_acc.append(accuracy.item()*100/batch_size)
        if counter%100==0:
            print("Epoch: {}/{}".format(e,epochs),
                         "\tIteration: {}".format(counter),
                         "\t\tTrain Loss: {:.3f}".format(loss.item()),
                         "\tTrain Accuracy: {:.2f}".format(accuracy.item()*100/batch_size))
            CNN_acc.append(accuracy.item()*100/batch_size)
        counter += 1
    train_loss_epoch.append(np.round(np.mean(train_loss), 3))
    train_acc_epoch.append(np.round(np.mean(train_acc), 3))     
    print("\tTrain Loss: {:.3f}".format(np.mean(train_loss)), "\tTrain Acc: {:.3f}".format(np.mean(train_acc)))

    
    with torch.no_grad():
        model.eval()
        val_acc = []
        val_loss = []
        for inputs, labels in val_dl:
            inputs_val, labels_val = inputs.cuda(), labels.cuda()
            logits_val,accuracy_val = model(inputs_val,labels_val)
            loss_val = criterion(logits_val,labels_val.float())
            val_acc.append(accuracy_val.item()*100/batch_size)
            val_loss.append(loss_val.item())
        val_loss_epoch.append(np.round(np.mean(val_loss), 3))
        val_acc_epoch.append(np.round(np.mean(val_acc), 3))
        print("\t\tVal Loss: {:.3f}".format(np.mean(val_loss)), "\t\tVal Acc: {:.3f}".format(np.mean(val_acc)))
        CNN_valacc.append(np.mean(val_acc))
        model.train()

    end_time = time.time()-start_time
    print("Time to train epoch: {0} s".format(end_time))
    time_epoch.append(np.round(end_time, 3))
    
write_to_csv(epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_acc_epoch, time_epoch)
    
with torch.no_grad():
    model.eval()
    test_acc = []
    test_loss = []
    for inputs, labels in test_dl:
        input_test, labels_test = inputs.cuda(), labels.cuda()
        logits_test,accuracy_test = model(input_test,labels_test)
        loss_test = criterion(logits_test,labels_test.float())
        test_acc.append(accuracy_test.item()*100/batch_size)
        test_loss.append(loss_test.item())
    print("Test Loss: {:.3f}".format(np.mean(test_loss)), "\tTest Acc: {:.3f}".format(np.mean(test_acc)))
    append_to_csv(epochs, np.round(np.mean(test_acc),3))

Epoch: 0/10 	Iteration: 0 		Train Loss: 0.850 	Train Accuracy: 20.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.560 	Train Accuracy: 74.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 0.540 	Train Accuracy: 70.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 0.521 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 0.392 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 0.417 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 600 		Train Loss: 0.372 	Train Accuracy: 84.00
Epoch: 0/10 	Iteration: 700 		Train Loss: 0.435 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 800 		Train Loss: 0.401 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 900 		Train Loss: 0.243 	Train Accuracy: 92.00
Epoch: 0/10 	Iteration: 1000 		Train Loss: 0.364 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 1100 		Train Loss: 0.338 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 1200 		Train Loss: 0.246 	Train Accuracy: 90.00
Epoch: 0/10 	Iteration: 1300 		Train Loss: 0.249 	Train Accuracy: 90.00
Epoc

Epoch: 6/10 	Iteration: 10500 		Train Loss: 0.246 	Train Accuracy: 88.00
Epoch: 6/10 	Iteration: 10600 		Train Loss: 0.139 	Train Accuracy: 96.00
Epoch: 6/10 	Iteration: 10700 		Train Loss: 0.143 	Train Accuracy: 96.00
Epoch: 6/10 	Iteration: 10800 		Train Loss: 0.281 	Train Accuracy: 90.00
Epoch: 6/10 	Iteration: 10900 		Train Loss: 0.204 	Train Accuracy: 88.00
Epoch: 6/10 	Iteration: 11000 		Train Loss: 0.313 	Train Accuracy: 82.00
Epoch: 6/10 	Iteration: 11100 		Train Loss: 0.283 	Train Accuracy: 92.00
Epoch: 6/10 	Iteration: 11200 		Train Loss: 0.359 	Train Accuracy: 86.00
Epoch: 6/10 	Iteration: 11300 		Train Loss: 0.175 	Train Accuracy: 94.00
Epoch: 6/10 	Iteration: 11400 		Train Loss: 0.135 	Train Accuracy: 98.00
Epoch: 6/10 	Iteration: 11500 		Train Loss: 0.281 	Train Accuracy: 88.00
Epoch: 6/10 	Iteration: 11600 		Train Loss: 0.195 	Train Accuracy: 92.00
	Train Loss: 0.219 	Train Acc: 91.002
		Val Loss: 0.338 		Val Acc: 88.527
Time to train epoch: 368.79587483406067 s
Epoch: 7